Lights Out!

In [ ]:
import numpy as np
import random

tablero = np.empty((5, 5))

for i in range(tablero.shape[0]):
    for j in range(tablero.shape[1]):
        tablero[i, j] = np.random.randint(0, 2)
tablero = tablero.astype(int)
copia = np.copy(tablero)


def generar_tablero_alcanzable():
    n1 = np.array([0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0])
    n2 = np.array([1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1])

    while True:
        tablero = np.empty((5, 5))

        for i in range(tablero.shape[0]):
            for j in range(tablero.shape[1]):
                tablero[i, j] = np.random.randint(0, 2)
        tablero = tablero.astype(int)
        
        b = tablero.reshape(25)
        
        bn1 = np.dot(b, n1) % 2
        bn2 = np.dot(b, n2) % 2

        tablero_alcanzable = (bn1 == 0 and bn2 == 0)

        if tablero_alcanzable:
            return tablero

tablero = generar_tablero_alcanzable()

def cambio_estado(celda):
    if celda == 0:
        return 1
    else:
        return 0

def modificar_celdas(tablero, fila, col):
    if fila >= 0 and fila <= 4 and col >= 0 and col <= 4:
        tablero[fila, col] = cambio_estado(tablero[fila, col])
    if fila-1 >= 0 and fila-1 <= 4 and col >= 0 and col <= 4:
        tablero[fila-1, col] = cambio_estado(tablero[fila-1, col])
    if fila >= 0 and fila <= 4 and col+1 >= 0 and col+1 <= 4:
        tablero[fila, col+1] = cambio_estado(tablero[fila, col+1])
    if fila >= 0 and fila <= 4 and col-1 >= 0 and col-1 <= 4:
        tablero[fila, col-1] = cambio_estado(tablero[fila, col-1])
    if fila+1 >= 0 and fila+1 <= 4 and col >= 0 and col <= 4:
        tablero[fila+1, col] = cambio_estado(tablero[fila+1, col])
    

def verificar(tablero):
    return np.all(tablero == 0)

In [ ]:
i = (np.identity(5, dtype=int))
nula = (np.zeros((5,5), dtype=int))
B = (np.array([[1,1,0,0,0],[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[0,0,0,1,1]]))
a = (np.zeros((5,5), dtype=int))
b = tablero.reshape(25)

fila1 = np.hstack([B, i, nula, nula, nula])
fila2 = np.hstack([i, B, i, nula, nula])
fila3 = np.hstack([nula, i, B, i, nula])
fila4 = np.hstack([nula, nula, i, B, i])
fila5 = np.hstack([nula, nula, nula, i, B])

A = np.vstack([fila1, fila2, fila3, fila4, fila5])

def eliminar_gauss(A, b):
    n = A.shape[0]
    Ab = np.column_stack((A, b)) #matriz ampliada

    for i in range(n):
        if Ab[i, i] == 0: #busco los pivotes
            for j in range(i+1, n):
                if Ab[j, i] == 1:
                    Ab[i], Ab[j] = Ab[j].copy(), Ab[i].copy()
                    break
            else:
                continue

        for j in range(i+1, n): #voy a escalonar la matriz
            if Ab[j, i] == 1:
                Ab[j] = (Ab[j] + Ab[i])%2
                
    x = np.zeros(n, dtype=int) #vector solución nulo
    for i in range(n-1, -1, -1):
        x[i] = Ab[i, -1]
        for j in range(i+1, n):
            x[i] = (x[i] + Ab[i, j] * x[j])%2

    return x

x = eliminar_gauss(A, b) #solución del sistema Ax=b

n1 = np.array([0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0])
n2 = np.array([1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1])

bn1 = np.dot(b, n1)%2
bn2 = np.dot(b, n2)%2

if bn1 ==0 and bn2 ==0:
    print("Ambos son ortogonales, existe solución")
    print("Solución: ", x)
else:
    print("Al menos uno no es ortogonal, no existe solución")

soluciones = np.array([x,(x+n1)%2,(x+n2)%2,(x+n1+n2)%2])
solucion = random.choice(soluciones)
solucion = np.reshape(solucion, (5, 5))

In [ ]:
##### CONSTANTES #####
import pygame
import sys

ANCHO = 800
ALTO = 600
DIM = 5
FPS = 40
BOT = 60  
LAT = 100  
CEL = min((ANCHO - 2 * LAT) // DIM, (ALTO - BOT - LAT) // DIM) 
amp = pygame.image.load('bulb.png') #cargar la imagen de la ampolleta
amp = pygame.transform.scale(amp, (CEL, CEL)) #la escalo para que quepa en la eclda

negro = (0,0,0)
blanco = (255,255,255)
colores = [pygame.Color("darkslategray4"), pygame.Color("darkslategray2")] #colores[0] apagado, colores[1] prendido
colores_sol = [pygame.Color("plum2"), pygame.Color("plum3")]

mostrando_solucion = False

In [ ]:
#función para dibujar el tablero en el que se juega

def d_tablero(ventana, tablero): 
    for f in range(DIM):
        for c in range(DIM):
            color = colores[tablero[f][c]]
            pygame.draw.rect(ventana, color, pygame.Rect(LAT + c * CEL, BOT + f * CEL, CEL, CEL))

    for f in range(DIM + 1):
        pygame.draw.line(ventana, negro, (LAT, BOT + f * CEL), (LAT + DIM * CEL, BOT + f * CEL), 2)  
        pygame.draw.line(ventana, negro, (LAT + f * CEL, BOT), (LAT + f * CEL, BOT + DIM * CEL), 2)  

    pygame.draw.rect(ventana, negro, pygame.Rect(LAT, BOT, DIM * CEL, DIM * CEL), 4)

In [ ]:
# función para poner una ampolleta en las celdas encendidas

def prender(ventana, tablero): 
    for f in range(DIM):
        for c in range(DIM):
            p = tablero[f][c]
            if p == 1: 
                ventana.blit(amp, (LAT + c * CEL, BOT + f * CEL))
                pygame.display.flip()

In [ ]:
#función para poder hacer un contador de movimientos

def movimientos(ventana, mov):
    fuente = pygame.font.Font("coolvetica rg.otf", 36) 
    movs = fuente.render(f"Movimientos: {mov}", True, negro)  
    ventana.blit(movs, (10, 10))

In [ ]:
#función para hacer un botón para reiniciar el tablero

def b_reiniciar(ventana, texto, x, y, ancho, alto): 
    fuente_b = pygame.font.Font("coolvetica rg.otf", 16) 
    boton = pygame.Rect(x, y, ancho, alto)
    pygame.draw.rect(ventana, negro, boton, 2)
    b_txt = fuente_b.render("Reiniciar Tablero", True, negro)
    ventana.blit(b_txt, (x + (ancho - b_txt.get_width()) // 2, y + (alto - b_txt.get_height()) // 2))
    return boton

In [ ]:
# función para dibujar sobre el tablero original, el tablero de la solución
# las estrellas simbolizan dónde se debe apretar, siguiendo el orden de izq a der, de arriba a abajo

def d_solucion(ventana, solucion): 
    for f in range(DIM):
        for c in range(DIM):
            if solucion[f][c] == 0:
                color = colores_sol[0]  # plum2 para celda de valor 0
                pygame.draw.rect(ventana, color, pygame.Rect(LAT + c * CEL, BOT + f * CEL, CEL, CEL))
            else:
                color = colores_sol[1]  # plum3 para celda de valor 1
                pygame.draw.rect(ventana, color, pygame.Rect(LAT + c * CEL, BOT + f * CEL, CEL, CEL)) # si es de valor 1, le pongo estrella
                ventana.blit(estrella, (LAT + c * CEL, BOT + f * CEL)) 

    for f in range(DIM + 1): #dibujo la rejilla y el borde para el tablero
        pygame.draw.line(ventana, negro, (LAT, BOT + f * CEL), (LAT + DIM * CEL, BOT + f * CEL), 2)  
        pygame.draw.line(ventana, negro, (LAT + f * CEL, BOT), (LAT + f * CEL, BOT + DIM * CEL), 2)  

    pygame.draw.rect(ventana, negro, pygame.Rect(LAT, BOT, DIM * CEL, DIM * CEL), 4)
    pygame.display.flip()

In [ ]:
# función para dibujar las estrellas en el tablero

def mostrar_estrellas(ventana, tablero_solucion): 
    for f in range(DIM):
        for c in range(DIM):
            p = solucion[f][c]
            if p == 1: 
                ventana.blit(estrella, (LAT + c * CEL, BOT + f * CEL))
    pygame.display.flip()

In [ ]:
# función para hacer el botón para mostrar la solución

def b_solucion(ventana, texto, x, y, ancho, alto):
    fuente_bs = pygame.font.Font("coolvetica rg.otf", 16)
    boton = pygame.Rect(x, y, ancho, alto)
    pygame.draw.rect(ventana, negro, boton, 2)
    bs_txt = fuente_bs.render("Mostrar Solución", True, negro)
    ventana.blit(bs_txt, (x + (ancho - bs_txt.get_width()) // 2, y + (alto - bs_txt.get_height()) // 2))
    return boton

In [ ]:
estrella = pygame.image.load('star.png') #cargo la foto de la estrella, y escalo igual que la ampolleta
estrella = pygame.transform.scale(estrella, (CEL, CEL))

#función para mostrar el tablero de la solución

def mostrar_solucion(ventana, solucion):
    global mostrando_solucion
    d_solucion(ventana, solucion)  
    pygame.display.flip()
    mostrando_solucion = True  
    pygame.time.set_timer(pygame.USEREVENT, 4000)  

In [ ]:
#función para devolver el tablero original luego de mostrar la solución, además de los botones y contador

def restaurar_tablero(ventana, tablero):
    d_tablero(ventana, tablero) 
    prender(ventana, tablero) 
    movimientos(ventana, mov)  
    b_rei = b_reiniciar(ventana, "Reiniciar", ANCHO - 180, 10, 120, 50)
    b_sol = b_solucion(ventana, "Solución", ANCHO - 180, 70, 120, 50)
    pygame.display.flip()
    pygame.time.set_timer(pygame.USEREVENT, 0)  
    global mostrando_solucion
    mostrando_solucion = False 

In [ ]:
esnupi = pygame.image.load('snoopy.jpeg') #cuando ganes el juego, aparece un snoopy bailando
esnupi = pygame.transform.scale(esnupi, (ANCHO - LAT, ALTO - BOT))

#función para mostrar la victoria

def victoria(ventana):
    ventana.fill(blanco)
    ventana.blit(esnupi, (0, 0))
    fuente = pygame.font.Font("Coffee Soda.ttf", 64)
    txt_v = fuente.render("¡Ganaste!", True, negro)
    ventana.blit(txt_v, (ANCHO // 2 - txt_v.get_width() // 2, ALTO // 2 - txt_v.get_height() // 2))
    pygame.display.flip()
    pygame.time.delay(2000)

In [ ]:
ventana = pygame.display.set_mode((ANCHO,ALTO))
ventana.fill("lightcyan2")
mov = 0 #contador
pygame.display.set_caption("Lights Out!")
pygame.display.flip()


def main():
    global mov, tablero, copia #necesito que sean globales, sino no puedo acceder
    pygame.init()
    run = True
    reloj = pygame.time.Clock()

    d_tablero(ventana, tablero) #dibujo el tablero
    prender(ventana, tablero) #pongo las ampolletas
    movimientos(ventana,mov) #dibujo el contador
    b_rei = b_reiniciar(ventana, "Reiniciar", ANCHO - 180, 10, 120, 50) #dibujo el botón
    b_sol = b_solucion(ventana, "Solución", ANCHO - 180, 70, 120, 50) #dibujo el botón

    pygame.display.flip() #actualizo

    while run:
        reloj.tick(FPS)
        ventana.fill("lightcyan2")
        
        for event in pygame.event.get(): #salir del juego
            if event == pygame.QUIT:
                run = False
                break

            if event.type == pygame.USEREVENT and mostrando_solucion: #muestro la solución por 4 segundos
                restaurar_tablero(ventana, tablero)

            if event.type == pygame.MOUSEBUTTONDOWN:
                x, y = event.pos
                if b_sol.collidepoint(x, y): #si se aprieta el botón solución
                    if not mostrando_solucion:
                        mostrar_solucion(ventana, solucion)

            if event.type == pygame.MOUSEBUTTONDOWN: #si se aprieta alguna celda
                x, y = event.pos
                if LAT <= x < LAT + DIM * CEL and BOT <= y < BOT + DIM * CEL:
                    col = (x - LAT) // CEL
                    fila = (y - BOT) // CEL
                    modificar_celdas(tablero, fila, col)
                    mov += 1
                    d_tablero(ventana, tablero)
                    prender(ventana, tablero)
                    movimientos(ventana,mov)
                    b_rei = b_reiniciar(ventana, "Reiniciar", ANCHO - 180, 10, 120, 50) #dibujo los botones siempre luego de hacer cualquier cosa
                    b_sol = b_solucion(ventana, "Solución", ANCHO - 180, 70, 120, 50) 

                    pygame.display.flip()

                    if verificar(tablero) == True: #si es que el tablero tiene puros 0, llamo a victoria
                        victoria(ventana)
                        tablero = np.copy(copia)
                        mov = 0 
                    else:
                        d_tablero(ventana, tablero)
                        prender(ventana, tablero)
                        movimientos(ventana, mov)
                        b_rei = b_reiniciar(ventana, "Reiniciar", ANCHO - 180, 10, 120, 50)
                        b_sol = b_solucion(ventana, "Solución", ANCHO - 180, 70, 120, 50) 

                        pygame.display.flip()
                
                if b_rei.collidepoint(x, y): #si se aprieta el botón reiniciar
                        tablero = np.copy(copia) 
                        mov = 0 
                        d_tablero(ventana, tablero)
                        prender(ventana, tablero)
                        movimientos(ventana,mov)
                        b_rei = b_reiniciar(ventana, "Reiniciar", ANCHO - 180, 10, 120, 50)
                        b_sol = b_solucion(ventana, "Solución", ANCHO - 180, 70, 120, 50) 
                        pygame.display.flip()

                if b_sol.collidepoint(x, y): #si se aprieta el botón solución
                    if not mostrando_solucion: 
                        mostrar_solucion(ventana, solucion)

                        d_tablero(ventana, tablero) 
                        prender(ventana, tablero) 
                        movimientos(ventana, mov)
                        b_rei = b_reiniciar(ventana, "Reiniciar", ANCHO - 180, 10, 120, 50)
                        b_sol = b_solucion(ventana, "Solución", ANCHO - 180, 70, 120, 50) 
                        pygame.display.flip()

                       
    pygame.quit()
    sys.exit()



main()